In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Simple Deep MNIST tutorial

In [40]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [41]:
# this allows operations that build the graph to be intermixed with the ones that run the graph
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.matmul(x, W) + b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y))

#train_step = tf.train.GradientDescentOptimizer(0.2).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cross_entropy)
init_op = tf.initialize_all_variables()
init_global = tf.global_variables_initializer()

In [42]:
# initialize variables
sess.run(init_global)
sess.run(init_op)

for _ in range(1000):
    batch = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1]})
    if _ % 100 == 0:
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

sess.close()

0.5404
0.9046
0.9106
0.9147
0.9206
0.9162
0.9235
0.9105
0.9181
0.9185
0.9171


# Multilayer Convolutional Network for MNIST

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# stride of one
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    #return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

# max pooling over 2x2 blocks
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


# if code is run multiple times in the same session, 
# one would like to remove previous graph structure;
# otherwise all history is preserved
tf.reset_default_graph()

# placeholders
with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784])
    
with tf.name_scope("desired"):
    y_ = tf.placeholder(tf.float32, [None, 10])

# the input x has to be reshaped to a 4d tensor, with second and third dimensions
# corresponding to image width and height, and the last - number of output channels
with tf.name_scope("reshape"):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

# convolve the image, add the bias, apply ReLU and max pool
# size is reduced from 28x28 to 14x14
with tf.name_scope("conv1"):
    # receptive map is 5x5
    # there are 32 feature maps
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    
with tf.name_scope("pool1"):
    h_pool1 = max_pool_2x2(h_conv1)

with tf.name_scope("conv2"):
    # second convolutional layer; there are 64 feature maps
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    # size is reduced from 14x14 to 7x7
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

with tf.name_scope("pool2"):
    h_pool2 = max_pool_2x2(h_conv2)

with tf.name_scope("fc1"):
    # fully conected layer with 1024 neurons
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    #W_fc1 = weight_variable([4 * 4 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    #h_pool2_flat = tf.reshape(h_pool2, [-1, 4*4*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

with tf.name_scope("dropout"):
    # dropout
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

with tf.name_scope("fc2"):
    # output layer
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

with tf.name_scope("output"):
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [6]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

with tf.name_scope("cost"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

with tf.name_scope("optimizer"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
with tf.name_scope("prediction"):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    # add accuracy to summary
    tf.summary.scalar("accuracy", accuracy)

# prepare summary variable(s) to be shown in TensorBoard
summary = tf.summary.merge_all()
    
# prepare TensorBoard graph
# graph_location = tempfile.mkdtemp()
graph_location = './models2_deep_mnist/'
print('Saving graph to: %s' % graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(tf.get_default_graph())

init_global = tf.global_variables_initializer()

with tf.Session() as sess:
    #with tf.device('/cpu:0'):
    sess.run(init_global)
    
    for i in range(2000):
        batch = mnist.train.next_batch(50)
        
        if i % 100 == 0:
            #train_accuracy = accuracy.eval(feed_dict={
            #    x: batch[0], y_: batch[1], keep_prob: 1.0})
            
            # update the summary
            #run_summary = sess.run(summary, feed_dict={
            #    x: batch[0], y_: batch[1], keep_prob: 1.0})
            
            run_summary, train_accuracy = sess.run([summary, accuracy], feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            train_writer.add_summary(run_summary, i)
            
            print('step %d, training accuracy %g' % (i, train_accuracy))
            
        sess.run(train_step, feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 0.5})
        #train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        
        
        
    print('test accuracy %g' % accuracy.eval(feed_dict={
                x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Saving graph to: ./models2_deep_mnist/
step 0, training accuracy 0.14
step 100, training accuracy 0.88
step 200, training accuracy 0.92
step 300, training accuracy 0.94
step 400, training accuracy 0.86
step 500, training accuracy 0.9
step 600, training accuracy 0.92
step 700, training accuracy 0.98
step 800, training accuracy 0.94
step 900, training accuracy 0.94
step 1000, training accuracy 0.96
step 1100, training accuracy 0.9
step 1200, training accuracy 0.98
step 1300, training accuracy 0.96
step 1400, training accuracy 1
step 1500, training accuracy 0.98
step 1600, training accuracy 0.96
step 1700, training accuracy 0.96
step 1800, training accuracy 0.98
step 1900, training accuracy 1
test accuracy 0.9755
